In [1]:
import random

In [2]:
file = open('Pokemon Emerald.sav', 'rb+')
ram = bytearray(file.read())

In [3]:
# Get the most recent game

def getStartingPoint(ram):
    game_a_index = int.from_bytes(ram[0xFFC:0x1000], byteorder='little')
    game_b_index = int.from_bytes(ram[0xEFFC:0xF000], byteorder='little')

    if (game_a_index > game_b_index and game_a_index != 0xFFFFFFFF):
        print(f'Game A is the most recent: {game_a_index}')
        return 0x0
    else:
        print(f'Game B is the most recent: {game_b_index}')
        return 0xE000

startingPoint = getStartingPoint(ram)

Game B is the most recent: 1


In [4]:
# Sort the partitions for better writing. Don't have to stich sections 5-13

def partition(ram, low, high, starting_point):
    start = starting_point + (0x1000 * ((low + high) // 2)) + 0xFF4
    end = starting_point + (0x1000 * ((low + high) // 2)) + 0xFF6
    pivot = int.from_bytes(ram[start:end], byteorder='little')
    i = low - 1
    j = high + 1
    while True:
        i += 1
        while int.from_bytes(ram[starting_point + (0x1000 * i) + 0xFF4:starting_point + (0x1000 * i) + 0xFF6], byteorder='little') < pivot:
            i += 1
        j -= 1
        while int.from_bytes(ram[starting_point + (0x1000 * j) + 0xFF4:starting_point + (0x1000 * j) + 0xFF6], byteorder='little') > pivot:
            j -= 1
        if i >= j:
            return j
        ram[starting_point + (0x1000 * i):starting_point + (0x1000 * (i + 1))], ram[starting_point + (0x1000 * j):starting_point + (0x1000 * (j + 1))] = ram[starting_point + (0x1000 * j):starting_point + (0x1000 * (j + 1))], ram[starting_point + (0x1000 * i):starting_point + (0x1000 * (i + 1))]



def quickSort(ram, starting_point):
    def _quickSort(ram, low, high, starting_point):
        if low < high:
            split_index = partition(ram, low, high, starting_point)
            _quickSort(ram, low, split_index, starting_point)
            _quickSort(ram, split_index + 1, high, startingPoint)
    _quickSort(ram, 0, 13, starting_point)

quickSort(ram, startingPoint)

In [6]:
# Max out player money

def maxOutMoney(ram, starting_point):
    security_key = int.from_bytes(ram[starting_point + 0xAC:starting_point + 0xB0], byteorder='little')
    ram[starting_point + 0x1490:starting_point + 0x1494] = (0xFFFFFFFF ^ security_key).to_bytes(4, byteorder='little')

maxOutMoney(ram, startingPoint)

In [7]:
# Give pokeballs

def givePokeballs(ram, starting_point):
    pokeballs = [0x1, 0x2, 0x3, 0x4, 0x5, 0x6, 0x7, 0x8, 0x9, 0xA, 0xB, 0xC]
    pokeball_ram = bytearray(64)
    i = 0
    for pokeball in pokeballs:
        pokeball_ram[i:i + 2] = pokeball.to_bytes(2, byteorder='little')
        pokeball_ram[i + 2:i + 4] = 0x63.to_bytes(2, byteorder='little')
        i += 4
    ram[starting_point + 0x1650:starting_point + 0x1690] = pokeball_ram

givePokeballs(ram, startingPoint)

In [8]:
# Give items

def giveItems(ram, starting_point):
    items = [0x13, 0x19, 0x25, 0x44, 0x54, 0x55, 0xB6]
    item_ram = bytearray(120)
    i = 0
    for item in items:
        item_ram[i:i + 2] = item.to_bytes(2, byteorder='little')
        item_ram[i + 2:i + 4] = 0x63.to_bytes(2, byteorder='little')
        i += 4
    ram[starting_point + 0x1560:starting_point + 0x15D8] = item_ram

giveItems(ram, startingPoint)

In [11]:
# Write checksums

def writeChecksums(ram, starting_point):
    section_sizes = [0xF2C, 0xF80, 0xF80, 0xF80, 0xF08, 0xF80, 0xF80, 0xF80, 0xF80, 0xF80, 0xF80, 0xF80, 0xF80, 0x7D0]
    for i in range(14):
        section_size = section_sizes[i]
        checksum = 0
        j = 0
        while j < section_size:
            bytes_start = starting_point + (0x1000 * i) + j
            bytes_end = starting_point + (0x1000 * i) + j + 4
            checksum += int.from_bytes(ram[bytes_start:bytes_end], byteorder='little')
            j += 4
        checksum = ((checksum & 0xFFFFFFFF) + ((checksum & 0xFFFFFFFF) >> 16)) & 0xFFFF
        checksum_start = starting_point + (0x1000 * i) + 0xFF6
        checksum_end = starting_point + (0x1000 * i) + 0xFF8
        ram[checksum_start:checksum_end] = checksum.to_bytes(2, byteorder='little')

writeChecksums(ram, startingPoint)

In [12]:
file.seek(0)
file.write(ram)

131072